环境交互：
通过 Gym 创建的环境有一些标准方法，用于与环境交互：

- env.reset(): 初始化环境，并返回初始状态。
- env.step(action): 在环境中执行动作 action，返回一个四元组 (next_state, reward, done, info)。
- env.render(): 用于可视化环境。
- env.close(): 关闭环境。

> 这是完成循环操作中的关键步骤

gym出现的原因：不同于监督学习那样需要的是数据集，强化学习需要的是运行任务所需的环境，研究人员需要拥有标准化的环境和模块化的强化学习代码，方便复用以及方便研究人员能够在相同的环境和条件下测试算法

## 基本代码

# 几个关键模块拓展部分

## 向量化代码

实际上环境提供的就是向量，那么多个环境训练的过程无非就是环境的数据量增加了，类似于监督学习中的训练数据变多了，那么向量化处理后可以扩大其中的数据，加速其中的训练过程

`SyncVectorEnv`

https://blog.csdn.net/wxc971231/article/details/127567643

In [6]:
# import gym
# #envs = gym.vector.make('CartPole-v1', num_envs=3, disable_env_checker=None)
# envs = gym.vector.make('MiniGrid-Empty-6x6-v0',
                       
#                        num_envs=3,
#                        disable_env_checker=None)


# observations, infos = envs.reset()

# print('observations: ', observations)
# print('infos: ', infos)


NameNotFound: Environment MiniGrid-Empty-6x6 doesn't exist. 

### e.g. 向量化环境

In [12]:
import gym
from gym.vector import SyncVectorEnv

# 定义要使用的环境
def make_env(env_name):
    def _init():
        return gym.make(env_name)
    return _init

# 创建向量化环境，这里以 "CartPole-v1" 为例，使用了4个并行环境
num_envs = 4
envs = SyncVectorEnv([make_env("CartPole-v1") for _ in range(num_envs)])

# 重置所有环境
obs = envs.reset()

# 执行一个随机动作，获取所有环境的状态、奖励、结束标志和其他信息
action = envs.action_space.sample()  # 这里返回的是一个动作数组
state, reward, done, truncated, info = envs.step(action)

# 打印返回的结果
print(f"Observations: {state}")
print(f"Rewards: {reward}")
print(f"Dones: {done}")


Observations: [[-0.02410083 -0.1900203   0.03743307  0.3293185 ]
 [-0.03077811  0.18353297  0.02024684 -0.3059583 ]
 [ 0.0419991   0.19929028  0.02122417 -0.3085007 ]
 [ 0.02799987 -0.22800377  0.04983997  0.3039089 ]]
Rewards: [1. 1. 1. 1.]
Dones: [False False False False]


## obs-离散

In [20]:
import numpy as np
from gym import spaces
observation_space = spaces.Box(low=0, high=10, shape=(1,), dtype=np.float32)

b = 10
c = observation_space + b
# 打印 observation_space 的类型
c
observation_space
print(type(observation_space))
print(type(c))


TypeError: unsupported operand type(s) for +: 'Box' and 'int'

In [21]:
import numpy as np
from gym import spaces
action_space = spaces.Discrete(2)

print(type(action_space))



<class 'gym.spaces.discrete.Discrete'>


## obs-连续

In [24]:
import numpy as np
import gym
from gym import spaces

class TemperatureControlEnv(gym.Env):
    def __init__(self):
        super(TemperatureControlEnv, self).__init__()
        self.target_temperature = 22.0  # 目标温度
        self.initial_temperature = 20.0  # 初始温度
        self.max_temperature_change = 5.0  # 最大温度变化幅度
        
        # 定义连续动作空间，范围为[-max_change, max_change]
        self.action_space = spaces.Box(low=-self.max_temperature_change, 
                                        high=self.max_temperature_change, 
                                        dtype=np.float32)
        
        # 定义状态空间，范围为实际的温度值
        self.observation_space = spaces.Box(low=0, high=100, dtype=np.float32)  # 假设温度范围为0到100
        self.state = self.initial_temperature  # 当前状态

    def reset(self):
        self.state = self.initial_temperature
        return np.array([self.state], dtype=np.float32)

    def step(self, action):
        # 根据动作调整温度
        self.state += action  # 动作影响状态
        self.state = np.clip(self.state, 0, 100)  # 确保温度在合理范围内
        
        # 计算奖励，越接近目标温度奖励越高
        reward = -abs(self.state - self.target_temperature)  # 奖励为负的绝对误差
        
        done = False  # 在此示例中不考虑结束条件
        
        return np.array([self.state], dtype=np.float32), reward, done, {}

    def render(self):
        print(f"Current temperature: {self.state:.2f}, Target temperature: {self.target_temperature:.2f}")

# 训练代码（简化示例）
env = TemperatureControlEnv()
for episode in range(10):
    state = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()  # 随机选择一个动作
        next_state, reward, done, _ = env.step(action)
        env.render()

KeyboardInterrupt: 

## state 复杂维度拓展

In [31]:
size = 6

observation_space = spaces.Dict({
    "agent": spaces.Box(low=0, high=size - 1, shape=(2,), dtype=int),
    "target": spaces.Box(low=0, high=size - 1, shape=(2,), dtype=int),
})

observation_space
# 将observation_space转换为一个字典
observation = observation_space.sample()
print(observation)

# 将字典转换为一个数组
observation = np.array([observation["agent"], observation["target"]])
print(observation)


OrderedDict([('agent', array([3, 1])), ('target', array([3, 5]))])
[[3 1]
 [3 5]]


## wrapper 使用

In [ ]:
from gym.wrappers import FilterObservation



## 完整的拓展环境部分例子

### 复杂的奖励函数

In [ ]:
def step(self, action):
    
    self.state += action
    
    target_state = 5 # 不过一般放在 init 里面
    reward = -abs(self.state - target_state)
    
    # 一般 done = True 就结束了
    done = self.state >= 10 or self.state <= -10
    
    info = {}
    
    return self.state, reward, done, info


In [ ]:
# 范数计算奖励部分

target_state = -np.linalgnorm(self.state - target_state)


#### 加权范数

In [ ]:
# 可以考虑不同维度的奖励计算的不同
# 类似于 Shuvo 里面的设置部分

weight = np.array([0.5, 0.2, 0.1, 0.1, 0.1])

reward = -np.linalg.norm(weight * (self.state - target_state)) 
